# 1) Projeto: API Open Weather Map

Este projeto de API Open Weather Map tem como objetivo retirar informações de tempo atuais de cidades Brasileiras!

Importando as bibliotecas necessárias para tratamento das informações

In [2]:
import json
import requests
import pandas as pd
import numpy as np
import xlsxwriter

Obtive um arquivo json com o nome de todas as cidades.
O arquivo json city_list.json foi obtido no próprio site http://bulk.openweathermap.org/sample/city.list.json.gz

Como este arquivo está zipado, baixei ele diretamente no mesmo diretório deste jupyter notebook e tratei ele a partir daí.

In [3]:
# Lendo arquivo de cidades do Open Weather
json_file = pd.read_json('city_list.json', encoding='utf-8')

# Salvando todos os registros de cidades brasileiras
json_file = json_file[json_file['country']=='BR']

# Salvando somente os nomes de cidades brasileiras
cities_list = list(json_file['name'])
cities_list

['Viana',
 'Sao Joao dos Inhamuns',
 'Solanea',
 'Sobradinho',
 'Federative Republic of Brazil',
 'Rio da Areia',
 'Estado do Parana',
 'Piraquara',
 'Atuba',
 'Olaria',
 'Estado de São Paulo',
 'Arujá',
 'Pôrto do Sapé',
 'Altamira',
 'Cardeal Mota',
 'Ôlho d’Água',
 'Distrito Federal',
 'Gama',
 'Guará',
 'Chavantina',
 'Anori',
 'Curuai',
 'Maracá',
 'Lapa',
 'Stio Caracol',
 'Sao Borja',
 'Guamaré',
 'Guararapes',
 'Grão Mogol',
 'Conde',
 'Vigia',
 'Trindade',
 'Timon',
 'Timbiras',
 'Sirinhaem',
 'Serra Talhada',
 'Sao Mateus do Maranhao',
 'Sao Luis',
 'Sao Domingos do Maranhao',
 'Santa Quiteria do Maranhao',
 'Santana do Ipanema',
 'Santa Luzia',
 'Santana',
 'Pombal',
 'Piracuruca',
 'Jatoba',
 'Pesqueira',
 'Penalva',
 'Pedra Branca',
 'Paulo Afonso',
 'Paraipaba',
 'Paracuru',
 'Matriz de Camaragibe',
 'Macau',
 'Macaiba',
 'Lajedo',
 'Lagoa do Itaenga',
 'Joao Camara',
 'Jaboatao',
 'Ipueiras',
 'Ipojuca',
 'Grajau',
 'Gloria do Goita',
 'Garanhuns',
 'Fortaleza',
 'Estrei

Tive que selecionar apenas 50 cidades do Brasil, pois o Jupyter não aguenta processar muitas cidades em uma iteração :(

Um ponto importante é que as cidades não estão divididas por estado, portanto, há nomes de cidades repetidos.

Este código faz a chamada via resquests.get() para o website do Open Weather Map, passando como parâmetro as cidades brasileiras selecionadas.

Como resultado final, teremos um dicionario para cada cidade, com informações de tempo. Tudo salvo na lista 'br_cities'.

In [5]:
br_cities = []
lista=cities_list[0:50]
for city in lista:
    try:
        url = 'http://api.openweathermap.org/data/2.5/weather?q={}&appid=3064e81432c51c7940eaa8dbe01ff65f'.format(city)
        response = requests.get(url)
        data = response.json()
        br_cities.append( data)
        print('city: {city} response code: {status}'.format(city=city, status=response))
    except:
        print('Can not retrieve weather information for the city: ', city)

city: Viana response code: <Response [200]>
city: Sao Joao dos Inhamuns response code: <Response [200]>
city: Solanea response code: <Response [200]>
city: Sobradinho response code: <Response [200]>
city: Federative Republic of Brazil response code: <Response [200]>
city: Rio da Areia response code: <Response [200]>
city: Estado do Parana response code: <Response [200]>
city: Piraquara response code: <Response [200]>
city: Atuba response code: <Response [200]>
city: Olaria response code: <Response [200]>
city: Estado de São Paulo response code: <Response [200]>
city: Arujá response code: <Response [200]>
city: Pôrto do Sapé response code: <Response [200]>
city: Altamira response code: <Response [200]>
city: Cardeal Mota response code: <Response [200]>
city: Ôlho d’Água response code: <Response [200]>
city: Distrito Federal response code: <Response [404]>
city: Gama response code: <Response [200]>
city: Guará response code: <Response [200]>
city: Chavantina response code: <Response [200

Imprimindo as informações de tempo das cidades selecionadas

In [6]:
br_cities

[{'coord': {'lon': -40.5, 'lat': -20.39},
  'weather': [{'id': 803,
    'main': 'Clouds',
    'description': 'broken clouds',
    'icon': '04d'}],
  'base': 'stations',
  'main': {'temp': 297.61,
   'pressure': 1018,
   'humidity': 83,
   'temp_min': 296.48,
   'temp_max': 299.26},
  'visibility': 10000,
  'wind': {'speed': 3.1, 'deg': 360},
  'clouds': {'all': 75},
  'dt': 1570214548,
  'sys': {'type': 1,
   'id': 8474,
   'message': 0.0109,
   'country': 'BR',
   'sunrise': 1570177246,
   'sunset': 1570221638},
  'timezone': -10800,
  'id': 3445153,
  'name': 'Viana',
  'cod': 200},
 {'coord': {'lon': -40.28, 'lat': -6},
  'weather': [{'id': 803,
    'main': 'Clouds',
    'description': 'broken clouds',
    'icon': '04d'}],
  'base': 'stations',
  'main': {'temp': 307.517,
   'pressure': 1008.07,
   'humidity': 25,
   'temp_min': 307.517,
   'temp_max': 307.517,
   'sea_level': 1008.07,
   'grnd_level': 954.71},
  'wind': {'speed': 3.62, 'deg': 127.566},
  'clouds': {'all': 63},
  'd

Definindo uma função 'redefine_dict()' que irá remodelar toda a estrutura de dados de tempo para cada cidade. Ao invés de conter tipos diferentes de valores (lista, dicionario e string), os valores serão somente do tipo string. Neste caso, as keys dos dicionarios encadeados farão parte da key do dicionario mãe. 

Conforme exemplo:
A estrutura do dicionario para a cidade de São Paulo é:

Após a transformação da função abaixo, ela ficará:

Definindo a função redefine_dict(), que recebe como argumento um dicionario cujos valores têm que ser do tipo list, dict ou string. 

Retorna um dicionario com valores do tipo string somente.

As subkeys não são perdidas, mas incorporadas à key 'mãe'.

In [7]:
def redefine_dict(dict_city):
    dict_temp = {}
    for key,value in dict_city.items():
        if isinstance(value, list):
            for sub_value in value:
                for sub_v_key, sub_v_value in sub_value.items():
                    dict_temp['_'.join([str(key), str(sub_v_key)])] = sub_v_value
        elif isinstance(value, dict):
            for sub_v_key, sub_v_value in value.items():
                dict_temp['_'.join([str(key), str(sub_v_key)])] = sub_v_value
        elif isinstance(value, str):
            dict_temp[key] = value
    return dict_temp

Aplicando a remodelação da estrutura do dicionario contendo as informações de tempo das cidades selecionadas, através da chamada à função 'redefine_dict'.

In [8]:
redefined_cities = []

for city in br_cities:
    redefined_cities.append(redefine_dict(city))

#imprimindo o primeiro elemento da lista, somente como amostra
print(redefined_cities[0])

{'coord_lon': -40.5, 'coord_lat': -20.39, 'weather_id': 803, 'weather_main': 'Clouds', 'weather_description': 'broken clouds', 'weather_icon': '04d', 'base': 'stations', 'main_temp': 297.61, 'main_pressure': 1018, 'main_humidity': 83, 'main_temp_min': 296.48, 'main_temp_max': 299.26, 'wind_speed': 3.1, 'wind_deg': 360, 'clouds_all': 75, 'sys_type': 1, 'sys_id': 8474, 'sys_message': 0.0109, 'sys_country': 'BR', 'sys_sunrise': 1570177246, 'sys_sunset': 1570221638, 'name': 'Viana'}


Salvando as listas de cidade e previsão do tempo atual em um DataFrame com o nome 'df_weather_brcities'

In [9]:
colunas = list(redefined_cities[0].keys())
df_weather_brcities = pd.DataFrame(redefined_cities, columns=colunas)
df_weather_brcities.head(15)

,coord_lon,coord_lat,weather_id,weather_main,weather_description,weather_icon,base,main_temp,main_pressure,main_humidity,...,wind_speed,wind_deg,clouds_all,sys_type,sys_id,sys_message,sys_country,sys_sunrise,sys_sunset,name
0,-40.50,-20.39,803.0,Clouds,broken clouds,04d,stations,297.610,1018.00,83.0,...,3.10,360.000,75.0,1.0,8474.0,0.0109,BR,1.570177e+09,1.570222e+09,Viana
1,-40.28,-6.00,803.0,Clouds,broken clouds,04d,stations,307.517,1008.07,25.0,...,3.62,127.566,63.0,NaN,NaN,0.0047,BR,1.570177e+09,1.570221e+09,Sao Joao dos Inhamuns
2,-35.54,-6.75,800.0,Clear,clear sky,01d,stations,301.417,1010.95,46.0,...,7.46,118.510,1.0,NaN,NaN,0.0073,BR,1.570176e+09,1.570220e+09,Solanea
3,-53.03,-29.41,500.0,Rain,light rain,10d,stations,297.617,1011.61,87.0,...,2.49,336.883,91.0,NaN,NaN,0.0071,BR,1.570180e+09,1.570225e+09,Sobradinho
4,-55.00,-10.00,803.0,Clouds,broken clouds,04d,stations,310.117,1007.24,26.0,...,1.80,96.652,60.0,NaN,NaN,0.0066,BR,1.570181e+09,1.570225e+09,Federative Republic of Brazil
5,-49.25,-28.57,500.0,Rain,light rain,10d,stations,293.017,1015.09,92.0,...,0.26,125.101,100.0,NaN,NaN,0.0059,BR,1.570179e+09,1.570224e+09,Rio da Areia
6,-51.00,-24.00,800.0,Clear,clear sky,01d,stations,307.717,1011.08,24.0,...,1.39,0.082,0.0,NaN,NaN,0.0073,BR,1.570180e+09,1.570224e+09,Estado do Parana
7,-49.06,-25.44,800.0,Clear,clear sky,01d,stations,300.020,1017.00,44.0,...,2.10,100.000,0.0,1.0,8346.0,0.0066,BR,1.570179e+09,1.570224e+09,Piraquara
8,-49.21,-25.39,800.0,Clear,clear sky,01d,stations,302.140,1017.00,44.0,...,2.10,100.000,0.0,1.0,8346.0,0.0062,BR,1.570179e+09,1.570224e+09,Atuba
9,-44.55,-20.06,800.0,Clear,clear sky,01d,stations,304.017,1012.93,27.0,...,1.87,44.155,6.0,NaN,NaN,0.0058,BR,1.570178e+09,1.570223e+09,Olaria


Salvando o DataFrame em um arquivo .csv

In [11]:
# salvando DataFrame em formato csv
df_weather_brcities.to_csv('BR_Cities_Weather.csv', encoding='utf-8')

# 2) Projeto: Webscraping Receita Federal

Este projeto tem como meta obter informações de empresas com base em seu CPNJ no site da Receita Federal

In [5]:
import json
import requests
import pandas as pd
import numpy as np
import xlsxwriter
from bs4 import BeautifulSoup

De acordo com o site Finance One, as top 11 fintechs brasileiras são:
1) nubank = 
2) bidu 
3) guia bolso 
4) picpay 
5) toro investimentos
6) creditas
7) neon
8) quinto andar
9) contabilizei
10) conta azul
11) banco original 

Dado isso, vamos inserir o CNPJ dessas empresas em uma lista.

Obs.: O site da Receita Federal bloqueia requisições maiores do que 3 pesquisas de CNPJ, portanto, redefini a lista para pesquisar somente 3 :( Chorei.

In [12]:
nubank = 18236120000158
bidu = '00459359000121'
guiabolso = 15674094000151
picpay = 16590302000105
toro_inv = 12455479000130
creditas = '03881423000156'
neon = 20855875000182
quintoandar = 16788643000181
contabilizei = 20182807000108
contaazul = 14793473000106
bancooriginal = 92894922000108

# pesquisando somente 3 empresas por motivo de bloqueio de IP, feito pelo site da Receita Federal
cnpjs = [nubank, bidu, guiabolso, picpay]

#cnpjs = [nubank, bidu, guiabolso, picpay, toro_inv, creditas, neon, quintoandar, contabilizei, contaazul, bancooriginal]


For loop para consultar no site da Receita Federal os CNPJs do ranking de top 11 fintechs brasileiras.

In [ ]:
top_11_fintechs = []
import time

for cnpj in cnpjs:
    try:
        #headers = get_user_agent()
        url = 'https://www.receitaws.com.br/v1/cnpj/{}'.format(cnpj)
        #response = requests.get(url,headers=headers, timeout=90000)
        response = requests.get(url,timeout=90000)
        data = response.json()
        top_11_fintechs.append(data)
        time.sleep(10)
        print('cnpj: {cnpj} response code: {status}'.format(cnpj=cnpj, status=response))
    except:
        print('Can not retrieve company information for the CNPJ: ', cnpj)

Can not retrieve company information for the CNPJ:  18236120000158


In [11]:
top_11_fintechs

[{'atividade_principal': [{'text': 'Administração de cartões de crédito',
    'code': '66.13-4-00'}],
  'data_situacao': '04/06/2013',
  'nome': 'NU PAGAMENTOS S.A.',
  'uf': 'SP',
  'telefone': '(11) 2039-0650 / () 0000-0000',
  'atividades_secundarias': [{'text': 'Atividades de intermediação e agenciamento de serviços e negócios em geral, exceto imobiliários',
    'code': '74.90-1-04'},
   {'text': 'Outras atividades de serviços prestados principalmente às empresas não especificadas anteriormente',
    'code': '82.99-7-99'},
   {'text': 'Suporte técnico, manutenção e outros serviços em tecnologia da informação',
    'code': '62.09-1-00'}],
  'qsa': [{'qual': '10-Diretor', 'nome': 'DAVID VELEZ OSORNO'},
   {'qual': '10-Diretor', 'nome': 'CRISTINA HELENA ZINGARETTI JUNQUEIRA'},
   {'qual': '10-Diretor', 'nome': 'GABRIEL HADDAD SILVA'},
   {'qual': '10-Diretor', 'nome': 'DENNIS MIN WANG'},
   {'qual': '10-Diretor', 'nome': 'DIEGO CERUTTI'},
   {'qual': '10-Diretor', 'nome': 'ADAM EDWARD

Definindo a função 'redefine_dict_2', que é quase igual a função anteriormente apresentada (redefine_dict).

O que muda: caso a key do dicionário seja igual a 'qsa', ela adiciona o valor inteiro (sem particionar o dict) para o dict_temp() que vai ser retornado no final da função. Isso foi feito pois esta informação não tem um padrão fácil de encontrar para redefini-la em novas colunas.

In [16]:
def redefine_dict_2(dict_city):
    dict_temp = {}
    for key,value in dict_city.items():
        if key == 'qsa':
            dict_temp[key] = value
        elif isinstance(value, list):
            for sub_value in value:
                for sub_v_key, sub_v_value in sub_value.items():
                    dict_temp['_'.join([str(key), str(sub_v_key)])] = sub_v_value
        elif isinstance(value, dict):
            for sub_v_key, sub_v_value in value.items():
                dict_temp['_'.join([str(key), str(sub_v_key)])] = sub_v_value
        elif isinstance(value, str):
            dict_temp[key] = value
    return dict_temp

Fazendo a chamada da função acima para a lista de dicionarios de informações de fintechs obtido na Receita Federal. 

In [17]:
fintechs_dict_formated = []
for fintech in top_11_fintechs:
    fintechs_dict_formated.append(redefine_dict_2(fintech))

Imprimindo as informações para ver o resultado

In [18]:
fintechs_dict_formated

[{'atividade_principal_text': 'Administração de cartões de crédito',
  'atividade_principal_code': '66.13-4-00',
  'data_situacao': '04/06/2013',
  'nome': 'NU PAGAMENTOS S.A.',
  'uf': 'SP',
  'telefone': '(11) 2039-0650 / () 0000-0000',
  'atividades_secundarias_text': 'Suporte técnico, manutenção e outros serviços em tecnologia da informação',
  'atividades_secundarias_code': '62.09-1-00',
  'qsa': [{'qual': '10-Diretor', 'nome': 'DAVID VELEZ OSORNO'},
   {'qual': '10-Diretor', 'nome': 'CRISTINA HELENA ZINGARETTI JUNQUEIRA'},
   {'qual': '10-Diretor', 'nome': 'GABRIEL HADDAD SILVA'},
   {'qual': '10-Diretor', 'nome': 'DENNIS MIN WANG'},
   {'qual': '10-Diretor', 'nome': 'DIEGO CERUTTI'},
   {'qual': '10-Diretor', 'nome': 'ADAM EDWARD WIBLE'},
   {'qual': '10-Diretor', 'nome': 'HUGH LIONEL STRANGE'},
   {'qual': '10-Diretor', 'nome': 'BRUNO MAGRANI DE SOUZA'},
   {'qual': '10-Diretor', 'nome': 'HENRIQUE CAMOSSA SALDANHA FRAGELLI'},
   {'qual': '10-Diretor', 'nome': 'ALESSANDRO DA COS

Transformando os dicionários em dataframes para serem salvos em arquivo

In [19]:
colunas = list(fintechs_dict_formated[0].keys())
df_top_fintechs = pd.DataFrame(fintechs_dict_formated, columns=colunas)
df_top_fintechs.head(15)

,atividade_principal_text,atividade_principal_code,data_situacao,nome,uf,telefone,atividades_secundarias_text,atividades_secundarias_code,qsa,situacao,...,fantasia,complemento,email,efr,motivo_situacao,situacao_especial,data_situacao_especial,capital_social,billing_free,billing_database
0,Administração de cartões de crédito,66.13-4-00,04/06/2013,NU PAGAMENTOS S.A.,SP,(11) 2039-0650 / () 0000-0000,"Suporte técnico, manutenção e outros serviços ...",62.09-1-00,"[{'qual': '10-Diretor', 'nome': 'DAVID VELEZ O...",ATIVA,...,,,,,,,,1368170511.45,True,True
1,"Corretores e agentes de seguros, de planos de ...",66.22-3-00,15/11/2003,BIDU CORRETORA DE SEGUROS E SERVICOS DE INTERN...,SP,(11) 3053-5566 / (11) 3053-5567,Consultoria em tecnologia da informação,62.04-0-00,[{'qual': '37-Sócio Pessoa Jurídica Domiciliad...,ATIVA,...,,,societario@jfgranja.com.br,,,,,52299826.00,True,True
2,Correspondentes de instituições financeiras,66.19-3-02,24/05/2012,GUIABOLSO FINANCAS CORRESPONDENTE BANCARIO E S...,SP,(11) 2373-8523 / (11) 2373-8523,"Outras sociedades de participação, exceto hold...",64.63-8-00,"[{'qual': '05-Administrador', 'nome': 'BENJAMI...",ATIVA,...,GUIABOLSO,ANDAR 9,financeiro@guiabolso.com.br,,,,,222839226.00,True,True


Salvando o data frame no arquivo Top_Fintechs_RF.csv

In [20]:
df_top_fintechs.to_csv('Top_Fintechs_RF.csv', encoding='utf-8')